# Preprocessing

In [ ]:
!pip install openpyxl
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
df = pd.read_excel("raw/eval.xls")
df.head()

,Review,Label
0,Japanese style Italian restaurant. Ikura Pasta...,POSITIVE
1,Definately visit this love'd their masala dosa,POSITIVE
2,I had a great experience at Meadesmoore! We or...,POSITIVE
3,Food was good! Service level was slightly belo...,POSITIVE
4,"Food, presentation and service were all top no...",POSITIVE


In [3]:
df.isnull().sum()

Review    0
Label     0
dtype: int64

In [4]:
# Function to lowercase the text
def lowercase(text):
    return text.lower()

# Function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to handle contractions
def handle_contractions(text):
    contractions = {
        "ain't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "I am",
        "I've": "I have",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as",
        "this's": "this is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "here's": "here is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
    }
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

# Function to tokenize the text
def tokenize_text(text):
    return word_tokenize(text)

# Function to remove stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

# Function to lemmatize the tokens
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

# Function to handle ellipses
def handle_ellipses(text):
    return text.replace('…', ' ')

# Function to handle apostrophes
def handle_apostrophes(text):
    return text.replace("’", " ")

In [5]:
def preprocess_text(text):
    text = lowercase(text)
    text = remove_emojis(text)
    text = handle_contractions(text)
    text = remove_punctuation(text)
    text = handle_ellipses(text)
    text = handle_apostrophes(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_tokens(tokens)
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [6]:
df['Reviews_Cleaned'] = df['Review'].apply(preprocess_text)

In [7]:
df.to_excel("test.xlsx",index=False)

# Classification

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

### TextBlob

In [ ]:
!pip install TextBlob

In [9]:
from textblob import TextBlob

In [10]:
def classify_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "POSITIVE"
    else:
        return "NEGATIVE"

In [11]:
df['Predicted_Label'] = df['Review'].apply(classify_sentiment)

precision = precision_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.8430610191199341
Recall: 0.815
F-measure: 0.811137960147047


In [12]:
# Apply sentiment classification to the 'Review' column
df['Predicted_Label'] = df['Reviews_Cleaned'].apply(classify_sentiment)

# Calculate evaluation metrics
precision = precision_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.8318681318681318
Recall: 0.802
F-measure: 0.7974424552429669


### VADER

In [13]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samanthatan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
sid = SentimentIntensityAnalyzer()

def classify_sentiment_vader(text):
    scores = sid.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'POSITIVE'
    elif scores['compound'] <= -0.05:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

df['Predicted_Label_VADER'] = df['Review'].apply(classify_sentiment_vader)

precision = precision_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.8741092135779043
Recall: 0.774
F-measure: 0.7961183977313009


In [15]:
sid = SentimentIntensityAnalyzer()

def classify_sentiment_vader(text):
    scores = sid.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'POSITIVE'
    elif scores['compound'] <= -0.05:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

df['Predicted_Label_VADER'] = df['Reviews_Cleaned'].apply(classify_sentiment_vader)

precision = precision_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label_VADER'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.8501896110768146
Recall: 0.727
F-measure: 0.7408013982759893


### SenticNet

In [16]:
from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/samanthatan/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [17]:
def calculate_sentiment_score(text):
    tokens = nltk.word_tokenize(text)
    positive_score = 0
    negative_score = 0
    for token in tokens:
        synsets = list(swn.senti_synsets(token))
        if synsets:
            pos_score = sum(s.pos_score() for s in synsets) / len(synsets)
            neg_score = sum(s.neg_score() for s in synsets) / len(synsets)
            positive_score += pos_score
            negative_score += neg_score
    if positive_score > negative_score:
        return 'POSITIVE'
    elif positive_score < negative_score:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

df['Predicted_Label_SenticNet'] = df['Review'].apply(calculate_sentiment_score)

precision = precision_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)


Precision: 0.8364016809973449
Recall: 0.8049999999999999
F-measure: 0.8031270256742675


In [18]:
def calculate_sentiment_score(text):
    tokens = nltk.word_tokenize(text)
    positive_score = 0
    negative_score = 0
    for token in tokens:
        synsets = list(swn.senti_synsets(token))
        if synsets:
            pos_score = sum(s.pos_score() for s in synsets) / len(synsets)
            neg_score = sum(s.neg_score() for s in synsets) / len(synsets)
            positive_score += pos_score
            negative_score += neg_score
    if positive_score > negative_score:
        return 'POSITIVE'
    elif positive_score < negative_score:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

df['Predicted_Label_SenticNet'] = df['Reviews_Cleaned'].apply(calculate_sentiment_score)

precision = precision_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
recall = recall_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')
f1 = f1_score(df['Label'], df['Predicted_Label_SenticNet'], labels=['NEGATIVE', 'POSITIVE'], average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.8171722113502935
Recall: 0.757
F-measure: 0.7497474747474747


### BERT

In [ ]:
!pip install transformers

In [19]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

In [20]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

label_encoder = LabelEncoder()
df['Encoded_Label'] = label_encoder.fit_transform(df['Label'])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
tokenized_texts = tokenizer(df['Review'].tolist(), padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokenized_texts)

predicted_labels = torch.argmax(outputs.logits, dim=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels.numpy())
predicted_labels_encoded = label_encoder.transform(predicted_labels)

precision = precision_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')
recall = recall_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')
f1 = f1_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.5714092426110225
Recall: 0.538
F-measure: 0.4768050226717824


In [22]:
tokenized_texts = tokenizer(df['Reviews_Cleaned'].tolist(), padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokenized_texts)

predicted_labels = torch.argmax(outputs.logits, dim=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels.numpy())
predicted_labels_encoded = label_encoder.transform(predicted_labels)

precision = precision_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')
recall = recall_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')
f1 = f1_score(df['Encoded_Label'], predicted_labels_encoded, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f1)

Precision: 0.6346290110335054
Recall: 0.633
F-measure: 0.6318864565310063
